In [1]:
import sys
import boto3
import json
import pandas as pd
from my_secrets import my_secrets

# Assignment 2: Part 1

In [2]:
###########################################
#                READ DATA                #
###########################################

raw_data = pd.read_csv('data/Corona_NLP_test.csv').tail(10)
mturk_data = pd.read_csv('./assignments_1-25.csv')
mturk_data['ATime'] = pd.to_datetime(mturk_data['AcceptTime'])
mturk_data['STime'] = pd.to_datetime(mturk_data['SubmitTime'])

f = open('results_1-25.json')
results = json.load(f)
res_df = pd.DataFrame.from_records(results)


In [3]:
# for res in results:
#     print(res['hit_id'] + " " + str(res['sentiment']))

res_dict = {
    "389A2A3052C7KXMI1CS5Z71AN5BC0I": 3,
    "3AJA9FLWTQSPU4N8W47CDHM96CKFIR": 5,
    "3X7837UUBRSP6ME3DTP8VCOWQS66JR": 4,
    "3J5XXLQDI05Z0YRFBG5TRXN6BOB3V3": 1,
    "3O0M2G5VDKWMLNXG8T5G9KQFYWO94E": 2,
    "37VUR2VJ7OJ7RM70XVW5DCKKSXHC1L": 4,
    "3AXFSPQOZ4SLPF7EEMSCPI19JR9FJ3": 2,
    "3TFJJUELTVJEG7PWGZWJIVUPK3VC22": 3,
    "3O2Y2UIUD4OG0ZDWW7SOQX1MYG9FK2": 1,
    "3TL87MO8D0JRK5VEH74P5XE1NGEFL5": 5,
}

res_df['HITId'] = res_df['hit_id'] 

In [4]:
mturk_data['GroundTruth'] = mturk_data.apply(lambda x: res_dict[x['HITId']], axis=1)
mturk_data['MinDiff'] = (mturk_data.STime - mturk_data.ATime) / pd.Timedelta(minutes=1)
mturk_data['SecDiff'] = (mturk_data.STime - mturk_data.ATime) / pd.Timedelta(seconds=1)

mturk_data.head()

,AssignmentId,WorkerId,HITId,AutoApprovalTime,AcceptTime,SubmitTime,Answer,ATime,STime,GroundTruth,MinDiff,SecDiff
0,3RYC5T2D7IOG00DEQW2SSNWJMLUPRE,A2IXK982JL0ECO,389A2A3052C7KXMI1CS5Z71AN5BC0I,2023-02-24 14:21:49-05:00,2023-01-25 14:21:01-05:00,2023-01-25 14:21:49-05:00,1,2023-01-25 14:21:01-05:00,2023-01-25 14:21:49-05:00,3,0.800000,48.0
1,3SLE99ER028NYXYVH8E4J6TTEYYZBW,A2JX2VIJWXHS4X,389A2A3052C7KXMI1CS5Z71AN5BC0I,2023-02-24 14:24:33-05:00,2023-01-25 14:24:13-05:00,2023-01-25 14:24:33-05:00,4,2023-01-25 14:24:13-05:00,2023-01-25 14:24:33-05:00,3,0.333333,20.0
2,3QAPZX2QNJ8W8ZS9UL81CEN32L3206,A2JX2VIJWXHS4X,3AJA9FLWTQSPU4N8W47CDHM96CKFIR,2023-02-24 14:26:06-05:00,2023-01-25 14:25:56-05:00,2023-01-25 14:26:06-05:00,4,2023-01-25 14:25:56-05:00,2023-01-25 14:26:06-05:00,5,0.166667,10.0
3,3JJVG1YBEQSPRJE6AK7OU2MYDZLB5W,A2CRQ0XPXN1P0K,3X7837UUBRSP6ME3DTP8VCOWQS66JR,2023-02-23 22:02:17-05:00,2023-01-24 22:02:12-05:00,2023-01-24 22:02:17-05:00,1,2023-01-24 22:02:12-05:00,2023-01-24 22:02:17-05:00,4,0.083333,5.0
4,3HMVI3QICYNMG96YJPJWZN7IMS41YP,A2JX2VIJWXHS4X,3X7837UUBRSP6ME3DTP8VCOWQS66JR,2023-02-24 14:25:38-05:00,2023-01-25 14:25:31-05:00,2023-01-25 14:25:38-05:00,4,2023-01-25 14:25:31-05:00,2023-01-25 14:25:38-05:00,4,0.116667,7.0


Based on the average of mturk responses, how accurate are the existing labels?

In [5]:
avg_df = mturk_data[['HITId', 'Answer', 'GroundTruth']]
avg_df.head(3)


,HITId,Answer,GroundTruth
0,389A2A3052C7KXMI1CS5Z71AN5BC0I,1,3
1,389A2A3052C7KXMI1CS5Z71AN5BC0I,4,3
2,3AJA9FLWTQSPU4N8W47CDHM96CKFIR,4,5


In [6]:
agg_df = avg_df.groupby("HITId")["Answer", "GroundTruth"].agg(["mean", "std", "max", "count"])
agg_df.columns = agg_df.columns.map('|'.join).str.strip('|')


/var/folders/60/3nzg3xnj7s54m11cbswy4sf80000gn/T/ipykernel_92965/8824565.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  agg_df = avg_df.groupby("HITId")["Answer", "GroundTruth"].agg(["mean", "std", "max", "count"])


In [7]:
agg_df['diff'] = (agg_df['Answer|mean'] - agg_df['GroundTruth|mean']).abs()
agg_df.merge(res_df, how='inner', on='HITId')[['tweet', 'HITId', "Answer|mean", "GroundTruth|mean", "diff"]].sort_values(by=['diff'], ascending=False)


,tweet,HITId,Answer|mean,GroundTruth|mean,diff
8,I and @ForestandPaper members are committed to...,3TL87MO8D0JRK5VEH74P5XE1NGEFL5,2.75,5.0,2.25
5,"@RicePolitics @MDCounties Craig, will you call...",3O0M2G5VDKWMLNXG8T5G9KQFYWO94E,3.50,2.0,1.50
6,Gov need to do somethings instead of biar je r...,3O2Y2UIUD4OG0ZDWW7SOQX1MYG9FK2,2.50,1.0,1.50
0,Meanwhile In A Supermarket in Israel -- People...,37VUR2VJ7OJ7RM70XVW5DCKKSXHC1L,3.00,4.0,1.00
2,@calebmealer @thebradfordfile @realDonaldTrump...,3AJA9FLWTQSPU4N8W47CDHM96CKFIR,4.00,5.0,1.00
9,"Even though the Law Library is closed, ALL sub...",3X7837UUBRSP6ME3DTP8VCOWQS66JR,3.00,4.0,1.00
1,You never eaten the pigs cat dog or food from ...,389A2A3052C7KXMI1CS5Z71AN5BC0I,2.50,3.0,0.50
4,"With Gov Hogan's announcement that all bars, r...",3J5XXLQDI05Z0YRFBG5TRXN6BOB3V3,1.50,1.0,0.50
7,Asst Prof of Economics @cconces was on @NBCPhi...,3TFJJUELTVJEG7PWGZWJIVUPK3VC22,3.50,3.0,0.50
3,Did you panic buy a lot of non-perishable item...,3AXFSPQOZ4SLPF7EEMSCPI19JR9FJ3,2.00,2.0,0.00


Which tasks had the highest STD DEV?

In [8]:
agg_df.merge(res_df, how='inner', on='HITId')[['tweet', 'HITId', "diff", "Answer|std"]].sort_values(by=['Answer|std'], ascending=False)


,tweet,HITId,diff,Answer|std
1,You never eaten the pigs cat dog or food from ...,389A2A3052C7KXMI1CS5Z71AN5BC0I,0.50,2.121320
8,I and @ForestandPaper members are committed to...,3TL87MO8D0JRK5VEH74P5XE1NGEFL5,2.25,2.061553
9,"Even though the Law Library is closed, ALL sub...",3X7837UUBRSP6ME3DTP8VCOWQS66JR,1.00,1.732051
0,Meanwhile In A Supermarket in Israel -- People...,37VUR2VJ7OJ7RM70XVW5DCKKSXHC1L,1.00,1.414214
3,Did you panic buy a lot of non-perishable item...,3AXFSPQOZ4SLPF7EEMSCPI19JR9FJ3,0.00,1.000000
5,"@RicePolitics @MDCounties Craig, will you call...",3O0M2G5VDKWMLNXG8T5G9KQFYWO94E,1.50,0.707107
6,Gov need to do somethings instead of biar je r...,3O2Y2UIUD4OG0ZDWW7SOQX1MYG9FK2,1.50,0.707107
7,Asst Prof of Economics @cconces was on @NBCPhi...,3TFJJUELTVJEG7PWGZWJIVUPK3VC22,0.50,0.707107
4,"With Gov Hogan's announcement that all bars, r...",3J5XXLQDI05Z0YRFBG5TRXN6BOB3V3,0.50,0.577350
2,@calebmealer @thebradfordfile @realDonaldTrump...,3AJA9FLWTQSPU4N8W47CDHM96CKFIR,1.00,NaN


Which tasks took the longest? Calculated on average (in minutes)


In [9]:
avg_df = mturk_data[['HITId', 'MinDiff']]
agg_df = avg_df.groupby("HITId")["MinDiff"].agg(["mean", "std", "max", "count"])
# agg_df.columns = agg_df.columns.map('|'.join).str.strip('|')
# agg_df.head()

agg_df.merge(res_df, how='inner', on='HITId')[['tweet', 'HITId', "mean"]].sort_values(by=['mean'], ascending=False)

,tweet,HITId,mean
5,"@RicePolitics @MDCounties Craig, will you call...",3O0M2G5VDKWMLNXG8T5G9KQFYWO94E,1.208333
6,Gov need to do somethings instead of biar je r...,3O2Y2UIUD4OG0ZDWW7SOQX1MYG9FK2,1.066667
3,Did you panic buy a lot of non-perishable item...,3AXFSPQOZ4SLPF7EEMSCPI19JR9FJ3,0.722222
1,You never eaten the pigs cat dog or food from ...,389A2A3052C7KXMI1CS5Z71AN5BC0I,0.566667
4,"With Gov Hogan's announcement that all bars, r...",3J5XXLQDI05Z0YRFBG5TRXN6BOB3V3,0.458333
8,I and @ForestandPaper members are committed to...,3TL87MO8D0JRK5VEH74P5XE1NGEFL5,0.416667
0,Meanwhile In A Supermarket in Israel -- People...,37VUR2VJ7OJ7RM70XVW5DCKKSXHC1L,0.366667
7,Asst Prof of Economics @cconces was on @NBCPhi...,3TFJJUELTVJEG7PWGZWJIVUPK3VC22,0.308333
2,@calebmealer @thebradfordfile @realDonaldTrump...,3AJA9FLWTQSPU4N8W47CDHM96CKFIR,0.166667
9,"Even though the Law Library is closed, ALL sub...",3X7837UUBRSP6ME3DTP8VCOWQS66JR,0.127778


Which workers worked the fastest? And what was the variability in their answers

In [10]:
avg_df = mturk_data[['WorkerId', 'MinDiff', "Answer"]]
agg_df = avg_df.groupby("WorkerId")["MinDiff", "Answer"].agg(["mean", "std", "max", "min", "count"])
agg_df.columns = agg_df.columns.map('|'.join).str.strip('|')
agg_df.sort_values(by=['MinDiff|mean'], ascending=False)

# agg_df.merge(res_df, how='inner', on='WorkerId')[['tweet', 'WorkerId', "mean"]].sort_values(by=['mean'], ascending=False)

/var/folders/60/3nzg3xnj7s54m11cbswy4sf80000gn/T/ipykernel_92965/2439504496.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  agg_df = avg_df.groupby("WorkerId")["MinDiff", "Answer"].agg(["mean", "std", "max", "min", "count"])


,MinDiff|mean,MinDiff|std,MinDiff|max,MinDiff|min,MinDiff|count,Answer|mean,Answer|std,Answer|max,Answer|min,Answer|count
WorkerId,,,,,,,,,,
A27W8C25KKKFXE,1.583333,NaN,1.583333,1.583333,1,2.000000,NaN,2,2,1
AW5RT9YS268VA,1.533333,NaN,1.533333,1.533333,1,2.000000,NaN,2,2,1
A2IXK982JL0ECO,0.744444,0.386341,1.100000,0.333333,3,1.666667,1.154701,3,1,3
AOSDB00WXVME4,0.657143,0.723793,1.966667,0.033333,7,2.428571,1.397276,4,1,7
A2JX2VIJWXHS4X,0.293333,0.166704,0.550000,0.116667,10,3.300000,0.948683,4,2,10
A6EEIASZ5KBPV,0.233333,NaN,0.233333,0.233333,1,5.000000,NaN,5,5,1
A2CRQ0XPXN1P0K,0.083333,NaN,0.083333,0.083333,1,1.000000,NaN,1,1,1
A2QHKSGU21S0GA,0.050000,NaN,0.050000,0.050000,1,1.000000,NaN,1,1,1


In [11]:
# Produce final dataset


# Assignment 2: Part 2

In [23]:
###########################################
#                READ DATA                #
###########################################

raw_data = pd.read_csv('data/Corona_NLP_chosen.csv').tail(10)
mturk_data = pd.read_csv('./assignments_1-30.csv')
mturk_data['ATime'] = pd.to_datetime(mturk_data['AcceptTime'])
mturk_data['STime'] = pd.to_datetime(mturk_data['SubmitTime'])

f = open('results_1-30.json')
results = json.load(f)
res_df = pd.DataFrame.from_records(results)
res_df


,tweet,hit_id
0,"#Covid_19 Went to the Grocery Store, turns out...",3VJ4PFXFKH1Z7410F5AYIJZT3UIUAV
1,"When youre stockpiling food & other supplies, ...",3MD8CKRQ0DHIS5CN5HS667FSMI8RJ4
2,"While you are stocking up, waiting for quarant...",3126F2F5GMXP5CAGEODQ4W2LGSOPEP
3,Two important phone numbers NYers should know....,3OLZC0DJ9X9EQ19YWJCGV3PVU9WIVC
4,"Remember, with all the media deflection storie...",3WKGUBL7TDGHMPYPXQ86ETHAYTTL4H
5,The beginning of a global recession! #Coronav...,3VQTAXTYOHF4QE6SLJAUDUJZLC3UBQ
6,Popped into my local supermarket after work. T...,38RHULDVAC9R5Q93MTHCH5891IYIWM
7,Small reminder: stock up on bar soap for your ...,3IJ95K7NEB6NM5UU44QM0AI2TT2NG8
8,Just did online shopping and first available s...,3OCZWXS7021ZI82PBVRGJLNOYPOL5M
9,@Disney What would it take for my family to pa...,3JYPJ2TAZW2HBQAC9ZXWMENLST3PF7


In [27]:
def sentiment_to_numeric(x):
    if x=='Extremely Negative': return 1
    if x=='Negative': return 2
    if x=='Neutral': return 3
    if x=='Positive': return 4
    if x=='Extremely Positive': return 5

raw_data['Answer'] = raw_data['Sentiment'].apply(sentiment_to_numeric)


,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment,Answer
0,14,44966,NaN,4/3/2020,"#Covid_19 Went to the Grocery Store, turns out...",Extremely Positive,5
1,18,44970,"Toronto, Ontario",5/3/2020,"When youre stockpiling food & other supplies, ...",Neutral,3
2,29,44981,NaN,7/3/2020,"While you are stocking up, waiting for quarant...",Negative,2
3,31,44983,"New York, New York",7/3/2020,Two important phone numbers NYers should know....,Negative,2
4,36,44988,"Dont ask for freedom, take it.",8/3/2020,"Remember, with all the media deflection storie...",Extremely Negative,1
5,50,45002,#NoNazis?? kaspum zaháv,9/3/2020,The beginning of a global recession! #Coronav...,Extremely Negative,1
6,198,45150,London,11/3/2020,Popped into my local supermarket after work. T...,Positive,4
7,204,45156,"Ithaca, NY",11/3/2020,Small reminder: stock up on bar soap for your ...,Extremely Positive,5
8,717,45669,London,12/3/2020,Just did online shopping and first available s...,Neutral,3
9,771,45723,"Logan, UT",12/3/2020,@Disney What would it take for my family to pa...,Positive,4
